# データ分析基盤を関数型言語の特性を活かして構築する

# なんで関数型言語でデータ分析か?
データ分析と言えば、pythonやRなどの言語が主流です。理由としては、おそらく、実装が簡単、ライブラリが豊富というのが主な理由であると思います。 
実際にpythonやRで機械学習のライブラリをす多少は叩いてみたこともありますが、初心者が軽く触ってもなんとなく、結果がでるのはとてもおもしろかったです。ただし、どうしてもなんとなく以上に進めない歯痒さや、ライブラリの中身がブラックボックスになっているもどかしさ、そして何より、コードの再利用、テスタビリティー、可読性が自分の至らなさもあってどうしても難しいところがありました。そこで、 **オブジェクト&関数型言語により出来る限りスクラッチで圏と関数、オブジェクト指向の概念を基に実装するチャレンジをすることで、より見晴らしの良いデータ分析の基盤を構築してみようと思いました。**  
そのため、この基盤を構築するにあたって、3つの概念をうまく組み合わせる必要があります。
- 圏や関数の数学的な概念
- データ分析で行われるデータ処理、機械学習、モデルの検証などの概念
- オブジェクト指向などの、プログラミングの概念

それぞれ3つの全く異なる分野ですが、数学とデータ分析はそれぞれの分野が互いに補完し合い、プログラミングにおいて有益なはたらきをします。
圏や関数（正確に言うと、関数は圏の概念の一部になりますが、わかりやすさのため、あえて分けています。）は「数学」という強固なフレームワークをもち、そのフレームワークのおかげで、自分で一部だけ実装すれば、定理に裏付けされたロジックにより、あとはよしなにやってくれるための基盤をつくることができます。(デザインパターンで言うところの、Templeteパターン、Factoryパターンの使いどころです。)  
データ分析は、データの変換と処理により構築されるので、そのような数学的基盤はとても有用です。数学のフレームワークをバックグラウンドに、**具体的に何をするかをアルゴリズムにより肉付けてきます。**    
数学（フレームワーク）とデータ分析（アルゴリズム）を仲介役として、関数型プログラミングによって実装する事で、**この２つの概念を結びつけ、再利用性と可読性に秀でたデータ分析基盤を構築するのが、この記事の目的です。** 
なお、プログラミング言語としては、Scala(ver2.11 ~)を採用します。この言語は関数型としての特性と、オブジェクト指向としての両方の性質をもつので、より、可読性、再利用性の高いプログラミングを行う事ができると期待できます。

# 動作環境
- Scala 2.11.8 or higher
- Java 1.8.0_25
- SBT 0.13 or higher
- JFreeChart 1.0.17
- Apache Commons Math library 3.5
- Indian Institute of Technology Bombay CRF 0.2
- LIBSVM 0.1.6
- Akka 2.3.8 or higher
- Apache Spark 2.1.0 or higher 

# データ分析と圏論、関数との相互作用
データ分析について、簡単にまとめ、改めて圏論、関数との相性について考えます。

## データ分析の区分
### 分類 (Classification)
履歴データから、分類するためのパターンを抽出する(ex xx度以上の体温の場合、風邪と分類する)

### 予測(Prediction)
履歴データを参照し、現在の状態から、別の状態を予測（推測）する（ex 大規模の患者データを参考に、現在の健康状態から疾病リスクを予測する）

### 最適化(Optimization)
全体最適を大規模のデータから構築する

### 回帰 (Regression)
データに沿った関数を提供する（連続的な分類モデルと同一視出来る）

上記のどのようなデータ分析も、大まかには以下のプロセスを通して構築されます

## データ分析の作業フロー

1. 問題の定義
2. データの入手
3. データのクリーニング（ノイズの除去、規格化など）
4. データのパターン分け（必要に応じて）
5. 特徴量の選択と、モデルの選択
6. モデルの学習と検証　
7. モデルの性能の向上


結局のところ、データ分析とは、 **何らかの観測データ[T]を最終的な分析結果[V]に変換しているにすぎないです。** この考え方は、 **圏論のフレームワークを基に実装する可能性を示唆してます**

# 圏
数学的には、圏の定義は以下です。

が、重要なのは、 **圏とは、型と矢印で構築される概念なんだな** ということです。いわばこれから作るデータの流れの**インフラ基盤を提供してくれる存在**という認識をもつことが出来ればと思います。

# Scalaでデータ分析を始める
圏論&関数型の特徴はデータ分析に応用できる  

圏論の立場において、プログラミング上、特に重要なのは、関手とMonadの概念である

1. 関手(Functor): データの型を別の変換する概念: Scalaではmapにより実装される
2. Monad: ラップされている型を、別の型に変換する概念。ただし変換された後の型は変改前と同じ型でラップされる: ScalaではunitとflatMapにより実装される

2 は1. の特殊な場合である


関手もMonadも圏の概念から出てきたものである。
圏Mは以下の条件をみたす構造体である

- Objects of some type {x e X, y Є Y, z Є Z, …}
- Morphisms or maps applied to these objects x Є X, y Є Y, f: x -› y
- Composition of morphisms f: x -› y, g: y -› z => g o f: x -› z

# 高階型(HIGHER KINDED TYPES)
高階型は型を抽象化することにより、既存の型から、新しい型を構築する。
以下のように、パラメータに型をもつトレイトを構築したとすると、

In [1]:
trait M[T] {  .  } 

: 

高階型は以下のように実装される

In [1]:
trait H[M[_]]; class H[M[_]] 

: 

例: 射（Monad）は高階型である

In [1]:
TODO

: 

## 高階型をどのようにデータ分析に応用するか?
例えば、機械学習による分類は、以下のようにペクトル同士の**内積**とみなせる

In [1]:
TODO

: 

これは、圏論の概念に変換すると、**余ベクトルによる、線形写像とみなすことができる**

In [1]:
TODO

: 

この概念をScalaにより実装する。  
高階型`Hom`共射と反射をもつ型として定義する

In [2]:
type Hom[T] = {
  type Right[X] = Function1[X,T] // Co-vector
  type Left[X] = Function1[T,X]   // Vector
 }

defined type Hom

## 共関手( covariant functor )
一般的に関手というとこれをさす。下記の反関手と区別するため、共関手と呼ばれる  
共関手を以下のように定義する
- If f: x -› y is a morphism on C then F(x) -› F(y) is also a morphism on C
- If id: x -› x is the identity morphism on C then F(id) is also an identity morphism on C
- If g: y -› z is also a morphism on C then F(g o f) = F(g) o F(f)

In [3]:
trait Functor[M[_]]{ def map[U,V](m: M[U])(f: U=>V): M[V] }

defined trait Functor

In [3]:
例:ここで、観測された値を、n次元のベクトル(T)型として、定義する

: 

In [4]:
trait ObsFunctor[T] extends Functor[(Hom[T])#Left] { self =>
  override def map[U,V](vu: Function1[T,U])(f: U =>V): 
    Function1[T,V] = f.compose(vu)
}

defined trait ObsFunctor

## 反関手（contravariant functor）
時により余関手(co-functor)と言われる事もある  
反射を以下のように定義する
- If f: x -› y is a morphism on C then F(y) -› F(x) is also a morphism on C
- If id: x -› x is the identity morphism on C then F(id) is also an identity morphism on C
- If g: y -› z is also a morphism on C then F(g o f) = F(f) o F(g)


In [5]:
trait CoFunctor[M[_]]{def map[U,V](m:M[U])(f:V=>U):M[V]}

defined trait CoFunctor

In [5]:
trait CoObsVector[T] extends CoFunctor[(Hom[T])#Right]{self =>
    override def map[U,V](vu:Function1[U,T])(f: U=>V):
    Function1[U,T]=f.andThen(vu)
}

: 

# Data Piplene
上に書いた。７つのワークフローに沿って、データの分析がおこなわれいく、そのデータの流れを構築するためのパイプラインをScalaで構造化していく

## モデルとは
- A scientific model seeks to represent empirical objects, phenomena, and physical processes in a logical and objective way.

- Models that are rendered in software allow scientists to leverage computational power to simulate, visualize, manipulate and gain intuition about the entity, phenomenon or process being represented.

モデルとは、 **特定のシステムから観測されるデータを説明するため**に使用される。対象のデータをを説明するためのパターンをモデルとして構築し、そこから、新たなパターンを構築する  
Scala(関数型言語)においては、関数と圏の概念により、数学的バックグラウンドに基づいたモデルを構築する

## 圏論の立場によるモデルの構築
数学的には、関数も圏の一部と見なせる、ため圏と関数を分けるのは良くないが、プログラミングの立場からは、その役割が異なる事から意図的に区別してまとめる。大まかには、**圏: 大まかな構造（フレームワーク）を提供、関数: 圏では運用上難しいところを小回りよく活用する**ように役割をかえてつかう
- データの操作および連鎖に関して、高度に抽象化されたフレームワークを提供する
- 非常に協力だが、反面、高度に抽象化されている故に、小回りが効かない

## モデルの実装
データ変換を行う以下のようなモデルを定義する

In [19]:
import scala.util.Try

trait Transform[T,A]{ // T:インプットする特徴量のデータ,A:アウトプットされるデータ
    self=>
    def |> :PartialFunction[T,Try[A]] // データのバリデーションを行うメソッド
    def map[B](f:A=>V):ITransform[T,B]= ??? // 
    def flatMap[B](f:A=>ITransform[T,B]):ITransform[T,B]= ???
    def andThen[B](tr:ITransform[A,B]):ITransform[T,B]= ??? // monadにおいて、定義が必須ではないが、変換を結合する際に有用
}

import scala.util.Try
defined trait ITransform

In [18]:
map,flatMap,andThenの

: 

In [21]:
import scala.util.Try

trait ITransform[T,A]{ // T:インプットする特徴量のデータ,A:アウトプットされるデータ
    self=>
    def |> :PartialFunction[T,Try[A]] // データのバリデーションを行うメソッド
    
    def map[B](f:A=>B):ITransform[T,B]= new ITransform[T,B]{
        override def |> =new PartialFunction[T,Try[B]] {
            override def isDefinedAt(t:T)=
            self.|>.isDefinedAt(t)
            override def apply(t:T):Try[B]=self.|>(t).map(f)
        }
    }
    
    def flatMap[B](f:A=>ITransform[T,B]):ITransform[T,B]= new ITransform[T,B]{
        override def |> =new PartialFunction[T,Try[B]] {
            override def isDefinedAt(t:T)=
            self.|>.isDefinedAt(t)
            override def apply(t:T):Try[B]=self.|>(t).flatMap(f(_).|>(t))
        }
    }
    
    def andThen[B](tr:ITransform[A,B]):ITransform[T,B]= new ITransform[T,B]{
        override def |> =new PartialFunction[T,Try[B]] {
          override def isDefinedAt(t:T)=
            self.|>.isDefinedAt(t) && tr.|>.isDefinedAt(self.|>(t).get)
            override def apply(t:T)=tr.|>(self.|>(t).get)
        }
    }
}

import scala.util.Try
defined trait ITransform

データ変換において、外部の何らかのコンテキストに依存する場合もある。そのため、外部から`config`を注入できるタイプのモデル `Transform`を構築する

In [20]:
trait Config
case class ConfigInt(iParam:Int) extends Config
case class ConfigDouble(fParam:Double) extends Config
case class ConfigArrayDouble(fParam:Double) extends Config

defined trait Config
defined class ConfigInt
defined class ConfigDouble
defined class ConfigArrayDouble

## 関数の立場によるモデルの構築
- 関数の特性により、より小回りが効くモデルを提供する

- Declare the variables relevant to the problem.
- Define a model (equation, algorithm, formulas…) as the solution to the problem.
- Instantiate the variables and execute the model to solve the problem.

## 変数の宣言
関数の特性により、モデルを構築するときの基盤を考える  

In [9]:
type V=Vector[Double]
trait F{val f:V=>V}
trait G{val g:V=>Double}

defined type V
defined trait F
defined trait G

In [6]:
これは、数学的記号で定義すると、

: 

In [6]:
f:R^n=>R^n, g:R^n=>R

: 

In [6]:
と書ける

: 

## モデルの定義

この２つを関数合成した関数`h=f◦g`を実装することを要請した`class H`は以下のように定義できる

In [10]:
class H{self:F with G=>def apply(v:V):Double=g(f(v)) }

defined class H

## モデルのインスタンス化
上記で抽象化したモデルをインスタンスとして、実現するために、実装をする

In [13]:
import java.lang._
val h=new H with F with G{
    val f=(v:V)=>v.map(Math.exp) // f:x=>exp(x)
    val g = (v:V)=>v.sum// g:x=>Σxi
}

import java.lang._
h: H with F with G = cmd12$$user$$anonfun$1$$anon$1@16fe12e0

## 圏と関数で実際のデータ処理を行う
上記のような変数の宣言=>モデルのインスタンス化までの過程をよりオブジェクト指向の特性を活かして活用していく。
Scaladでは、`trait`により、**ロジックを注入していく事で、データ分析のワークフローの過程に沿って、動的にモジュールを置換する事が可能になる**  
そのため、ここからは上記での圏の概念と関数の概念によるモデリングに加え、オブジェクト指向の考え方を活用するため[MinmalCakePattern](TODO)を使用する

データの前処理において
- Sampling to extract a sample from raw data
- Normalization to normalize the sampled data over [0, 1]
- Aggregation to aggregate or reduce the data:

の三つのプロセスを行う必要があるとする

上で定義したデータ処理プロセスを`ITransform`により構築する

In [26]:
trait UsesSmapling[T,A]{val sampler:ITransform[T,A]}
trait UsesNomalization[T,A]{val nomalizer:ITransform[T,A]}
trait UsesAggregation[T,A]{val aggregator:ITransform[T,A]}

defined trait UsesSmapling
defined trait UsesNomalization
defined trait UsesAggregation

データ処理をそれぞれのフェイズに分解した。これらを組み合わせる事で、ワークフローを構築する

In [ ]:
class Workflow[T,U,V,W] extends 

In [27]:
trait ToDouble[T] { def apply(t: T): Double }
  implicit val str2Double = new ToDouble[String] {
     def apply(s: String): Double = s.toDouble
  }

defined trait ToDouble
str2Double: AnyRef with $user.ToDouble[String] = cmd26$$user$$anonfun$1$$anon$1@227a5f35

# まとめ
- 圏と関数でデータの流れを制御するパイプラインのインフラ基盤を構築した
- そのインフラ基盤にデータ分析のアルゴリズムを注入した
- さらにオブジェクト指向により、そのパイプラインの構築やメンテナンスを容易にした

ことが、できました。とはいえ、まだデータの前処理の段階なので、次回は単純ベイズモデルを例に、モデルの構築から、モデルの精度検証までまとめます

# リファレンス
- [共変、反変](https://ufcpp.wordpress.com/2010/04/19/%E5%85%B1%E5%A4%89%E6%80%A7%EF%BC%88covariance%EF%BC%89%E5%8F%8D%E5%A4%89%E6%80%A7%EF%BC%88contravariance%EF%BC%89%E3%81%AE%E8%A9%B1/)

# Tips

The notation for the view bound, T <% Double, is being deprecated in Scala 2.11 and higher. The declaration class A[T <% Float] is the short notation for class A[T](implicit f: T => Float).

    class MinMax[T <: AnyVal](val values: Vector[T])(implicit f :T=>Double) {

      case class ScaleFactors(low: Double, high: Double, ratio: Double)


      private var scaleFactors = None[ScaleFactors]


      private val range = (Double.MaxValue, -Double.MaxValue)

      // valuesのなかで最小な値をと最大な値を抽出する
      protected val minMax = values.foldLeft(range) { (mM, x) => {
        val min = mM._1
        val max = mM._2
        (if (x < min) x else min, if (x > max) x else max)
      }
      }
        
ここで、fが存在しないとTをDoubleに変換できる保証がないので、コンパイルが通らない x> でエラーが出る。よってfが必要        

In [1]:
val xt=Array(1,2,3,4,5)
val period=3
val (first, second) = xt.splitAt(period)

xt: Array[Int] = Array(1, 2, 3, 4, 5)
period: Int = 3
first: Array[Int] = Array(1, 2, 3)
second: Array[Int] = Array(4, 5)

In [2]:
val slider = xt.take(xt.size - period) zip second

slider: Array[(Int, Int)] = Array((1, 4), (2, 5))

In [3]:
sealed abstract class DataField(val value:Int)

  case class StockMarketDataField(column:Int)extends DataField(column)
  val DATE, OPEN, HIGH, LOW, CLOSE, VOLUME, ADJ_CLOSE= 0 to 6 match {case x=>StockMarketDataField(_)}


defined class DataField
defined class StockMarketDataField
DATE: $user.StockMarketDataField.type = StockMarketDataField
OPEN: $user.StockMarketDataField.type = StockMarketDataField
HIGH: $user.StockMarketDataField.type = StockMarketDataField
LOW: $user.StockMarketDataField.type = StockMarketDataField
CLOSE: $user.StockMarketDataField.type = StockMarketDataField
VOLUME: $user.StockMarketDataField.type = StockMarketDataField
ADJ_CLOSE: $user.StockMarketDataField.type = StockMarketDataField